
based off of article: 
http://mnemstudio.org/path-finding-q-learning-tutorial.htm

In [1]:
import numpy as np
#create rooms
adj_mat = {
    0: [4],
    1: [3,5],
    2: [3],
    3: [1,2,4],
    4: [0,3,5],
    5: [1,4,5]
}

#rewards. 100 points for reaching room 5. -1 denotes
#pairs without routes
rewards = np.array([[-1,-1,-1,-1,0,-1],
                    [-1,-1,-1,0,-1,100],
                    [-1,-1,-1,0,-1,-1],
                    [-1,0,0,-1,0,-1],
                    [0,-1,-1,0,-1,100],
                    [-1,0,-1,-1,0,100]])

#random room rate, to influence the room selection to the best learned so far
lr = 0.7

#gamma influences how much impact future reward have in comparison to 
#past/current reward.
gamma = 0.8

#update q function
def update_q(state, action, debug=False):
    #get list of actions that can be done from next state
    next_actions = adj_mat[action]

    #calculate q-table
    q_mat[state, action] = rewards[state, action] +\
                           gamma * np.max([q_mat[action, i] 
                                          for i in next_actions])
    if debug:
        print q_mat
        
def find_best_action(state):
    #get best action value for this state. np.argmax only returns the first occurence
    #so we have a different approach
    #action = q_mat[state].argmax(axis=0)
    
    #get indices of maximum values
    max_val = np.max(q_mat[state])
    max_indices = np.where(q_mat[state] == max_val)
    
    #get possible actions
    possible_actions = adj_mat[state]
    
    #choose random value from the intersection of the two
    return np.random.choice(np.intersect1d(possible_actions, max_indices))
    

    
def normalize_matrix(matrix):
    return matrix/float(np.max(matrix))

def test_algo(start_room):
    status = 1 #as long as room 5 is not found
    max_steps = 20
    step = 20
    while status:
        next_room = q_mat[start_room].argmax(axis=0)
        print 'start_room:', start_room
        print 'next_room:', next_room
        start_room = next_room
        print '\n'
        step += 1
        if next_room == 5:
            status = 0
        if step == max_steps:
            break
        

In [2]:
#init q_matrix
q_mat = np.zeros((6,6))

#loop for learning the q_Table
num_steps = 10
epochs = 500
max_steps = 60

for epoch in range(epochs):
    status = 1 #while room 5 haven't been found

    #get random room to start
    curr_room = np.random.randint(0,6)
    step = 0
    #traverse until hit the goal
    while status:
        #if fall here, use the best route found so far
        #to learn the q_table
        if np.random.random() < lr:
            action = find_best_action(curr_room)
            update_q(curr_room, action)
        else: #get a new room
            possible_actions = adj_mat[curr_room]
            action = np.random.choice(possible_actions)
            update_q(curr_room, action)
        curr_room = action            
        step += 0
        
        #if current room is 5, found goal
        if curr_room == 5:
            status = 0
        if step == max_steps:
            print 'max steps reached'
            break
q_mat = normalize_matrix(q_mat)

In [3]:
q_mat

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.8       ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.63990302,  0.        ,
         1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.64      ,  0.        ,
         0.        ],
       [ 0.        ,  0.79996677,  0.512     ,  0.        ,  0.8       ,
         0.        ],
       [ 0.63995495,  0.        ,  0.        ,  0.64      ,  0.        ,
         1.        ],
       [ 0.        ,  0.79961889,  0.        ,  0.        ,  0.79998523,
         0.99998154]])

In [4]:
test_algo(3)

start_room: 3
next_room: 4


start_room: 4
next_room: 5




## Neural network implementation of Q-learning

In [228]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation


In [229]:
def is_action(state, action):
    cur_room = np.where(state == 1)[0][0]
    if action not in adj_mat[cur_room]:
        #impossible action. 
        return False
    else:
        return True
    
def get_reward(state):
    cur_room = np.where(state==1)[0][0]
    if cur_room == 5:
        return 10
    else:
        return -1
        
def init_state(seed=None):
    state = np.zeros(6)
    if seed is None:
        rand_room = np.random.randint(0,6)
    else:
        rand_room = seed
    state[rand_room] = 1
    return state

def test_network(model, debug=False):
    max_steps = 15
    for i in range(0, 6):
        state = init_state(i)
        print 'starting room:', i
        step = 0
        while True:
            step += 1
            q_vals = model.predict(state.reshape(1,6))
            action = np.argmax(q_vals)
            print 'start_room:', np.argmax(state)
            print 'action:', action

            if step > max_steps:
                print 'max steps reached\n'
                break
            state = init_state(action)
            
            reward = get_reward(state)
            
            if reward != -1:
                print 'done\n'
                break

In [230]:
model = Sequential()

#1 hot encoding of rooms
model.add(Dense(128, input_shape=(6,)))
model.add(Activation('relu'))

#output layer. 6 nodes for the 6 rooms
model.add(Dense(6))
model.add(Activation('softmax'))

rms = RMSprop()
model.compile(loss='mse', optimizer='sgd')


In [231]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 128)               896       
_________________________________________________________________
activation_75 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_76 (Dense)             (None, 6)                 774       
_________________________________________________________________
activation_76 (Activation)   (None, 6)                 0         
Total params: 1,670
Trainable params: 1,670
Non-trainable params: 0
_________________________________________________________________


In [232]:
from IPython.display import clear_output

#variables
epochs = 1000
max_steps = 20#15
epsilon = 0.8
gamma = 0.7

verbose=0
debug = False
if debug:
    epochs = 2

for epoch in range(epochs):
    #set up room vector encoding
    state = init_state()
    step = 0
    if debug:
        print '\n'
        print 'init:', np.argmax(state)
    while True:
        if debug:
            print 'current state:', state
        
        step += 1
        #get the next action
        qval = model.predict(state.reshape(1,6))
        if debug:
            print 'initial qval:', qval
        
        if np.random.random() < epsilon:
            action = np.argmax(qval)
        
        else:
            action = np.random.randint(0,6)
        
        if debug:
            print 'action:', action
        #check if valid action
        if not is_action(state, action):
            if debug:
                print 'bad target'
            output = np.zeros((1,6)) #adjust shape for network input
            output = qval
            output[0][action] = -5 #this action shouldn't happen
            
            #fit the model to learn this choice
            model.fit(state.reshape(1,6), output,
                     epochs=1, verbose=0)
        
        else: #good next action
            if debug:
                print 'good target'
            #create new state to calculate Q(s', a)
            new_state = np.zeros(6)
            new_state[action] = 1
            if debug:
                print 'new state:', new_state
            #get reward
            reward = get_reward(new_state)
            
            #get maxQ of next state
            new_q = model.predict(new_state.reshape(1,6))
            max_q = np.max(new_q)
            
            #create target variable
            output = np.zeros((1,6))
            output = qval
            if reward <= 0: #non-terminal state
                update = reward + (gamma * max_q)
            else:
                update = reward
            output[0][action] = update
            
            #fit the desired output to the model
            model.fit(new_state.reshape(1,6), output, 
                     epochs=1, verbose=verbose)
            state = new_state
            
            if reward > 0: #found terminal room
                break
        if debug:
            print 'new target:', output
            new_q = model.predict(state.reshape(1,6))
            print 'new q_val:', new_q
            print '-----------'
        #clear_output(wait=True)
        #check if terminal room reached, or max_steps reached
        if step > max_steps:
            if debug:
                print 'max steps reached'
            break

In [233]:
state = np.array([1,0,0,0,0,0])
model.predict(state.reshape(1,6))

array([[ 0.11752021,  0.16640402,  0.17932314,  0.17418927,  0.1814322 ,
         0.18113114]], dtype=float32)

In [234]:
test_network(model)

starting room: 0
start_room: 0
action: 4
start_room: 4
action: 5
done

starting room: 1
start_room: 1
action: 5
done

starting room: 2
start_room: 2
action: 3
start_room: 3
action: 4
start_room: 4
action: 5
done

starting room: 3
start_room: 3
action: 4
start_room: 4
action: 5
done

starting room: 4
start_room: 4
action: 5
done

starting room: 5
start_room: 5
action: 5
done

